<a href="https://colab.research.google.com/github/vijayrgopu/apache_beam/blob/master/5_Combine_Transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install --quiet apache_beam[gcp]

     |████████████████████████████████| 3.4MB 1.4MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 225kB 14.4MB/s 
     |████████████████████████████████| 59.2MB 75kB/s 
     |████████████████████████████████| 1.2MB 42.8MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 92kB 10.6MB/s 
     |████████████████████████████████| 143kB 44.7MB/s 
     |████████████████████████████████| 235kB 43.9MB/s 
     |████████████████████████████████| 122kB 37.8MB/s 
     |████████████████████████████████| 174kB 41.1MB/s 
     |████████████████████████████████| 112kB 44.1MB/s 
ERROR: pydrive 1.3.1 has requirement oauth2client>=4.0.0, but you'll have oauth2client 3.0.0 which is incompatible.
ERROR: google-api-python-client 1.7.12 has requirement httplib2<1dev,>=0.17.0, but you'll have httplib2 0.12.0 which is incompatible.
ERROR: chainer 6.5.0 has requir

In [0]:
import apache_beam as beam

In [0]:
class AverageFn(beam.CombineFn):
  def create_accumulator(self):
    print('inside create')
    return (0.0, 0)

  def add_input(self, sum_count, input):
    (sum, count) = sum_count
    print('add_input : sum_count {0}'.format(sum_count))
    print('add_input : input {0}'.format(input))
    return (sum+input, count+1)

  def merge_accumulators(self, accumulators):
    print('merge_accumulators : input {0}'.format(accumulators))
    print('merge_accumulators : zip_acc {0}'.format(zip(*accumulators)))
    sums, counts = zip(*accumulators)
    print('merge_accumulators : sums {0}'.format(sums))
    print('merge_accumulators : counts {0}'.format(counts))
    return sum(sums), sum(counts)

  def extract_output(self,sum_count):
    print('extract_output : {0}'.format(sum_count))
    sum,count = sum_count
    return sum/count if count else float('NaN')  

with beam.Pipeline() as p1:

  small_sum = (
      p1
      | beam.Create([15,5,7,7,9,23,13,5])
      | beam.CombineGlobally(AverageFn())
      | beam.io.WriteToText('data/combine')
  )


inside create
extract_output : (0.0, 0)


inside create
add_input : sum_count (0.0, 0)
add_input : input 15
add_input : sum_count (15.0, 1)
add_input : input 5
add_input : sum_count (20.0, 2)
add_input : input 7
add_input : sum_count (27.0, 3)
add_input : input 7
add_input : sum_count (34.0, 4)
add_input : input 9
add_input : sum_count (43.0, 5)
add_input : input 23
add_input : sum_count (66.0, 6)
add_input : input 13
add_input : sum_count (79.0, 7)
add_input : input 5
merge_accumulators : input [(84.0, 8)]
merge_accumulators : zip_acc <zip object at 0x7ff42b7579c8>
merge_accumulators : sums (84.0,)
merge_accumulators : counts (8,)
extract_output : (84.0, 8)


In [0]:
!head -20 data/combine-00000-of-00001

9.333333333333334
